In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Group

In [ ]:
all_cones = Table.read_table('cones.csv')
#all_cones
cones = all_cones.drop('Color').exclude(5)
cones

In [ ]:
cones.group('Flavor')

In [ ]:
cones.group('Flavor', sum)
#help(cones.group)

In [ ]:
cones.where('Flavor', are.equal_to('chocolate')).column('Price')

In [ ]:
sum(cones.where('Flavor', are.equal_to('chocolate')).column('Price'))

In [ ]:
sum(cones.where('Flavor', are.equal_to('strawberry')).column('Price'))

In [ ]:
cones.group('Flavor', max)

In [ ]:
cones.group('Flavor', list)

Which NBA team payed their players the most?

In [ ]:
nba = Table.read_table('nba_salaries.csv').relabeled(3, 'SALARY')
nba

Keep only team and salary columns for analysis

In [ ]:
teams_and_money = nba.select('TEAM', 'SALARY')
teams_and_money

In [ ]:
teams_and_money.group('TEAM', sum)

In [ ]:
teams_and_money.group('TEAM', sum).sort(1, descending=True).barh('TEAM')

In [ ]:
nba.group('POSITION')

In [ ]:
positions_and_money = nba.select('POSITION', 'SALARY')
positions_and_money.group('POSITION', np.mean)

In [ ]:
nba.group('POSITION', np.mean)

## Group by multiple columns

In [ ]:
all_cones

In [ ]:
all_cones.group('Flavor')

In [ ]:
all_cones.group(['Flavor', 'Color'])

In [ ]:
all_cones.group(['Flavor', 'Color'], min)

## Challenge question

Which NBA teams spent the most on their starters in 2016?
* Each team has one starter per position
* Assume the starter for a team & position is the player with the highest salary on that team in that position

In [ ]:
nba

In [ ]:
starters = ...
starters

In [ ]:
...

## Pivot tables

In [ ]:
all_cones.pivot('Flavor', 'Color')

In [ ]:
all_cones.pivot('Flavor', 'Color', values='Price', collect=list)

In [ ]:
all_cones.group(['Flavor', 'Color'], min)

In [ ]:
nba.pivot('POSITION', 'TEAM')

In [ ]:
nba.pivot('POSITION', 'TEAM', values='SALARY', collect=max)

## Joins

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price']).with_rows([
    ['Milk Tea', 'Panda Tea Lounge', 4],
    ['Espresso', 'Gimme',  2],
    ['Latte',    'Gimme',  3],
    ['Espresso', "Cafe Gola",   2]
])
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(25, 50, 5),
    'Location', make_array('Panda Tea Lounge', 'Gimme', 'Gimme')
)
discounts

In [ ]:
t = drinks.join('Cafe', discounts, 'Location')
t

In [ ]:
t.with_column('Discounted', t.column(2) * (1 - t.column(3)/ 100))

In [ ]:
two = drinks.join('Cafe', drinks)
two

In [ ]:
two.with_column('Total', two.column('Price') + two.column('Price_2'))

## Bikes

From Citi Bike System Data website: https://www.citibikenyc.com/system-data.

In [ ]:
citi = Table.read_table('citibike_nyc_201707_250000.csv')
citi

In [ ]:
trips = citi.relabeled("start station name", "start")\
            .relabeled("end station name", "end")\
            .select("start", "end")\
            .with_column("duration", citi.column("tripduration")/60)
trips

How long do people generally borrow a bike for?

In [ ]:
trips.hist("duration")

In [ ]:
longest = trips.column("duration").max()
print("Longest trip took", longest/60, "hours.")

In [ ]:
commute = trips.where('duration', are.below(50))
commute.hist('duration', bins=150, unit='minute')

Where did people pick up bikes?

In [ ]:
starts = commute.group('start').sort('count', descending=True)
starts

* West St & Chambers St is near Stuyvesant High School, Borough of Manhattan Community College, and another ferry terminal, on the Hudson greenway bike trail
* 12 Ave & W 40 St is a ferry terminal on the Hudson greenway bike trail
* Pershing Square North is across from Grand Central Station

In [ ]:
commute.pivot('start', 'end')

In [ ]:
shortest = commute.group(["start", "end"], min)
shortest

Which stations can you get to fastest from Grand Central Station ('Pershing Square North')?

In [ ]:
from_gc = shortest.where("start", are.containing('Pershing Square North')).sort(2)
from_gc

## Maps

In [ ]:
stations = citi.relabeled("start station name", "name")\
                .relabeled("start station latitude", "lat")\
                .relabeled("start station longitude", "lon")\
                .select("name", "lat", "lon")
stations

How many trips started at each station?

In [ ]:
unique_stations = stations.group(["name", "lat", "lon"])
unique_stations

Map it!

In [ ]:
Marker.map_table(unique_stations.select("lat", "lon", "name"))

In [ ]:
Circle.map_table(unique_stations.select("lat", "lon", "name"), color='blue', radius=10)

In [ ]:
blue_stations = unique_stations.with_columns(
    "color", np.full(unique_stations.num_rows, "blue"),
    "radius", np.round(unique_stations.column("count")/100))
blue_stations

In [ ]:
Circle.map_table(blue_stations.select("lat", "lon", "name", "color", "radius"))

How long does it take to get to any other station from grand central?

In [ ]:
from_gc

In [ ]:
gc_dest = from_gc.join("end", unique_stations, "name")
gc_dest

In [ ]:
color_from_gc = gc_dest.with_columns(
    "color", np.full(gc_dest.num_rows, "blue"),
    "radius", gc_dest.column("duration min"))
color_from_gc

In [ ]:
Circle.map_table(color_from_gc.select("lat", "lon", "end", "color", "radius"))

About how long do people spend on trips from each station?

In [ ]:
stations = citi.relabeled("start station name", "name")\
                .relabeled("start station latitude", "lat")\
                .relabeled("start station longitude", "lon")\
                .select("name", "lat", "lon")\
                .with_column("duration", citi.column("tripduration")/60)
stations

In [ ]:
stations.group(["name", "lat", "lon"], np.median)

In [ ]:
def round_median(duration, interval=5):
    """Round the numbers in the array duration to bins of size interval"""
    return np.round(np.median(duration) / interval, 0) * interval

In [ ]:
round_median([10,15,5])

In [ ]:
round_median([11,15,5])

In [ ]:
round_median([11,14,16])

In [ ]:
round_median([21,22,5])

In [ ]:
duration_by_station = stations.group(["name", "lat", "lon"], round_median)
duration_by_station

In [ ]:
duration_by_station = duration_by_station.relabeled("duration round_median", "duration")
duration_by_station

In [ ]:
duration_by_station.group('duration')

In [ ]:
colors = duration_by_station.group('duration')\
                        .where('duration', are.below(30))\
                        .with_column('color', 
                make_array('blue', 'green', 'purple', 'red', 'orange'))
colors

In [ ]:
colored = duration_by_station.join('duration', colors)\
                             .select('lat', 'lon', 'name', 'color')
colored

In [ ]:
Marker.map_table(colored)